In [4]:
from lyricsgenius import Genius
from collections import Counter
import re
from nltk.tokenize import TreebankWordTokenizer
import string
import pandas as pd
import requests
from tqdm.notebook import tqdm
import pickle
import os
import requests
import re
from bs4 import BeautifulSoup
from time import time
from utils import *
import unidecode


path = r'C:\Users\chris\Documents\GitHub\cs4300sp2021-rad338-jsh328-rpp62-cmc447\sample_data/'
# path = r'C:\Users\Ronnie\Documents\INFO4300\cs4300sp2021-jsh328-rad338-rpp62-cmc447\sample_data'

tokenizer = TreebankWordTokenizer()
punct = set(string.punctuation)
punct.update({"''", "``", "’"})
# genius = Genius('bVEbboB9VeToZE48RaiJwrnAGLz8VbrIdlqnVU70pzJXs_T4Yg6pdPpJrTQDK46p')
# genius.verbose = True

In [27]:
class GetLyrics():
    def __init__(self, df, processed_songs = None):
        self.genius_key = 'LSWwGG8PlhpA8wtGYNrhacvWSZTQ5QjqWuwiGx-SXvlxdQkS1EmFpq7Vd5x2dq6c'
        self.df = df
        self.song_lyrics = dict()
        if processed_songs:
            self.processed_songs = processed_songs
        else:
            self.processed_songs = set()


    def get_lyrics(self):
        start_time = time()

        genius = Genius(self.genius_key, verbose=False, retries = 5)
        
        artists_not_found, songs_not_found, songs_found = 0, 0, 0
        index = 0
        
        gb = self.df.groupby('primary_artist')
        pbar = tqdm(total = len(gb), desc = "0")
        for artist_name, subset in gb:
            artist_obj = genius.search_artist(artist_name, max_songs=0)
            if artist_obj is None:
                print(f"{artist_name} not found")
                artists_not_found += 1
            else:
                for _, row in subset.iterrows():
                    index += 1
                    uri = row['track_id']
                    if uri not in self.processed_songs and uri not in self.song_lyrics:
                        song_name = strip_name(row['track_name'])
                        song_obj = artist_obj.song(song_name)
                        if song_obj is None or not match(artist_name, song_obj.artist):
                            songs_not_found += 1
                        else:
                            lyrics = re.sub(r'[\(\[].*?[\)\]]', '', song_obj.to_text().lower())
                            lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])
                            tokens = [t for t in tokenizer.tokenize(lyrics) if t not in punct]
                            cnt = Counter(tokens)
                            self.song_lyrics[uri] = cnt
                            songs_found += 1 
                if index != 0:
                    success_rate = round(songs_found / index, 4)
                    pbar.set_description(f"{index}; {success_rate}", refresh=True)
            

#                     if index % 50 == 0:
#                         time_dif = time() - start_time
#                         songs_per_second = round(index / time_dif, 4)
#                         print(f'{songs_found} songs found, {artists_not_found} artists not found, {songs_not_found} songs not found, {index} songs processed in {time_dif} seconds, rate: {songs_per_second}, success_rate: {success_rate}')
            pbar.update(1)
        return self.song_lyrics

In [22]:
df = pd.read_csv(path + "SpotifyAudioFeaturesApril2019.csv")
df = df.loc[(df.instrumentalness < 0.5) & (df.popularity > 60)].reset_index(drop = True) # drop less popular and mostly instrumental songs
df['primary_artist'] = [unidecode.unidecode(row['artist_name'].split(",")[0]) for _, row in df.iterrows()]
# df = df.sort_values(by = 'popularity', ascending = False).iloc[:1000].reset_index(drop = True)
df = df.sort_values(by = 'popularity', ascending = False).iloc[2000:].reset_index(drop = True)
processed_lyrics = pickle.load(open(path + 'processed_lyrics.pkl', 'rb'))

In [28]:
scraper = GetLyrics(df, processed_lyrics)

In [29]:
lyrics = scraper.get_lyrics()

0:   0%|          | 0/1815 [00:00<?, ?it/s]

Banda La Ejecutiva de Mazatlan Sinaloa not found
Banda Rancho Viejo De Julio Aramburo La Bandononona not found
Brown Noise Baby not found
Cornelio Vega y su Dinastia not found
DJ Juan Cuba not found
H.I.N.P (Hot Issue of Ntl. Producers) not found
Lucca e Mateus not found
Missy & Blonde not found
Tito Torbellino Jr not found
Tranquility Spree not found
Zertox MO not found
Zibra Inguzu not found
dj 6rb not found


In [38]:
l1 = pickle.load(open(path + 'top1000_lyrics.pkl', 'rb'))
l2 = pickle.load(open(path + 'top1000_2000_lyrics.pkl', 'rb'))

In [39]:
l3 = {k:v for k,v in lyrics.items()}

In [40]:
l3.update(l1)

In [41]:
l3.update(l2)

In [42]:
len(l3)

5374

In [43]:
pickle.dump(l3, open(path + 'top_lyrics.pkl', 'wb'))

In [36]:
pickle.dump(processed_lyrics.union(set(lyrics.keys())), open(path + "processed_lyrics.pkl", 'wb'))

In [30]:
pickle.dump(lyrics, open(path + 'top2000_end_lyrics.pkl', 'wb'))

In [31]:
len(lyrics)

3477

In [33]:
len(processed_lyrics.union(set(lyrics.keys())))

5374

In [34]:
5374/5905

0.9100762066045724

In [32]:
temp = pd.read_csv(path + "SpotifyAudioFeaturesApril2019.csv")
len(temp.loc[(temp.instrumentalness < 0.5) & (temp.popularity > 60)]) # drop less popular and mostly instrumental songs


5905

# ----------------------------------DEPRECATED----------------------------------

In [43]:
df1_sample = df1.sample(n = 5000, random_state = 1)

In [44]:
df1_sample

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
62741,Rikko Dunez,35x6RlfDH87X4u1cgIKubI,Talkin',0.334000,0.563,232699,0.5270,0.000000,4,0.141,-9.419,0,0.2810,129.861,4,0.147,0
35566,Jim Rystron,4Qg4a6CbWV7U0GIqzkeqId,Walkin' in the Parlor,0.983000,0.594,62798,0.0286,0.000950,2,0.200,-21.146,1,0.0815,98.037,4,0.977,2
30314,D. LYNCH,5m6rUoikFmlviz0u1kZ9UH,Don't Breathe,0.113000,0.881,254473,0.9260,0.000000,0,0.115,-6.086,0,0.2470,99.953,4,0.354,24
32055,Beatrice Markus,6uWcz9DKYqaGuANysINgc1,Burstin' Out,0.007120,0.580,189092,0.8670,0.004290,1,0.333,-5.950,0,0.0555,146.028,4,0.358,1
18932,POLIÇA,0UkB0ISxcNoOnEh5sUMD1g,Agree,0.736000,0.586,201747,0.6210,0.000382,9,0.150,-8.082,0,0.0307,102.989,4,0.456,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21968,Taysav,7EQtd9bbrrhnjFfabqxge9,Once Upon A Time,0.342000,0.630,186514,0.8320,0.000000,7,0.325,-4.292,0,0.2920,165.408,4,0.735,21
14431,The Lonely Biscuits,1JiPojrxU3xlHdUosquh1L,Talk About,0.118000,0.683,201613,0.5800,0.000000,11,0.119,-7.945,1,0.0472,128.032,4,0.687,34
32211,June,5NDIseC2L12LTHoO7Zl328,Interlude (feat. Trae Tha Truth),0.965000,0.566,44319,0.8750,0.251000,0,0.239,-7.648,1,0.1500,84.913,1,0.618,14
94808,Jeremy Zucker,3PMsgFdVtCeTgjzE8X9TTs,better off - filous remix,0.164000,0.782,176347,0.5640,0.006890,8,0.182,-7.752,0,0.2260,101.058,4,0.448,69


In [45]:
df1_sample.to_csv(path + r"\sample5000_SAFApril2019.csv", index = False)

In [46]:
# df = pd.read_csv(path + r"\sample_scraped_data200.csv")

In [47]:
s = genius.search_song("Circles", "Post Malone", get_full_info = False).to_text().lower()

In [48]:
s = genius.search_song("Circles", "Post Malone", get_full_info = False)

In [49]:
def match(a, b):
    return (a in b) or (b in a)

def get_lyrics(song_name, song_artist):
    while True:
        try:
            s = genius.search_song(song_name, song_artist, get_full_info = False)
            break
        except:
            pass
    if s:
        query_name, query_artist = song_name.lower(), song_artist.lower()
        returned_name, returned_artist = s.title.lower(), s.artist.lower()

        if match(query_artist, returned_artist):
            return s.to_text().lower()

In [50]:
def get_all_lyrics(df, uri_colname = 'uri', trackname_colname = 'name', artist_colname = 'artist'):
    """
    df: DataFrame; uri, name, and artist must be columns in the dataframe
    
    returns a dictionary where key = song uri and value = Counter of tokenized lyrics
    """
    cnt_dict = dict() #dict(uri: Counter(lyric tokens))
    pbar = tqdm(total=len(df))
    for _, row in df.iterrows():
        uri, name, artists = row[[uri_colname, trackname_colname, artist_colname]]
        
        
        for s in ["-", "(", "feat."]:
            ix = name.find(s)
            if s != -1:
                name = name[:ix].strip()
            
        artist = artists.split(",")[0]
        lyrics = get_lyrics(name, artist)
        if lyrics:
            lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
            tokens = [t for t in tokenizer.tokenize(lyrics) if t not in punct]
            cnt = Counter(tokens)
            cnt_dict[uri] = cnt
        pbar.update(1)
    return cnt_dict

In [51]:
# lyrics_dict1 = get_all_lyrics(df1_sample, 'track_id', 'track_name', 'artist_name')
# with open(path + r'\sample5000_SAFApril2019_lyrics.pkl', 'wb') as f:
#     pickle.dump(lyrics_dict1, f)

In [52]:
len(lyrics_dict1)

NameError: name 'lyrics_dict1' is not defined

In [61]:
class GetLyrics():
    
#     def __init__(self, spotify_client_id, spotify_client_secret, user_id, playlist_id, genius_key):
#         self.spotify_client_id = spotify_client_id
#         self.spotify_client_secret = spotify_client_secret
#         self.user_id = user_id
#         self.playlist_id = playlist_id
#         self.genius_key = genius_key
        
#     def get_playlist_info(self):
#         token = SpotifyClientCredentials(client_id=self.spotify_client_id, client_secret=self.spotify_client_secret).get_access_token()
#         sp = spotipy.Spotify(token)
#         playlist = sp.user_playlist_tracks(self.user_id, self.playlist_id)
#         self.playlist = playlist
#         return self.playlist
    
#     def get_track_names(self):
#         track_names = []
#         for song in range(len(self.playlist['items'])):
#             track_names.append(self.playlist['items'][song]['track']['name'])
#         self.track_names = track_names
#         return self.track_names
    
#     def get_track_artists(self):
#         track_artists = []
#         for song in range(len(self.playlist['items'])):
#             track_artists.append(self.playlist['items'][song]['track']['artists'][0]['name'])
#         self.track_artists = track_artists
#         return self.track_artists
    def __init__(self, df):
        self.genius_key = 'bVEbboB9VeToZE48RaiJwrnAGLz8VbrIdlqnVU70pzJXs_T4Yg6pdPpJrTQDK46p'
        self.df = df
        
    def request_song_info(self, track_name, track_artist):
        self.track_name = track_name
        self.track_artist = track_artist
        base_url = 'https://api.genius.com'
        headers = {'Authorization': 'Bearer ' + self.genius_key}
        search_url = base_url + '/search'
        data = {'q': track_name + ' ' + track_artist}
        response = requests.get(search_url, data=data, headers=headers)
        self.response = response
        return self.response

    def check_hits(self):
        json = self.response.json()
        remote_song_info = None
        for hit in json['response']['hits']:
            if self.track_artist.lower() in hit['result']['primary_artist']['name'].lower():
                remote_song_info = hit
                break
        self.remote_song_info = remote_song_info
        return self.remote_song_info
    
    def get_url(self):
        song_url = self.remote_song_info['result']['url']
        self.song_url = song_url
        return self.song_url
    
    def scrape_lyrics(self):
        page = requests.get(self.song_url)
        html = BeautifulSoup(page.text, 'html.parser')
        lyrics1 = html.find("div", class_="lyrics")
        lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
        if lyrics1:
            lyrics = lyrics1.get_text()
        elif lyrics2:
            lyrics = lyrics2.get_text()
        elif lyrics1 == lyrics2 == None:
            lyrics = None
        return lyrics

    def get_lyrics(self):
        #playlist = GetLyrics.get_playlist_info(self)
        GENIUS_API_TOKEN='bVEbboB9VeToZE48RaiJwrnAGLz8VbrIdlqnVU70pzJXs_T4Yg6pdPpJrTQDK46p'
        token = GENIUS_API_TOKEN
        song_lyrics = []
        genius = Genius(token, verbose=False)
        # for i in range(len(self.track_names)):
        artists_not_found, songs_not_found, songs_found = 0, 0, 0
        start_time = time()
        index = 0
        for i, row in self.df.iterrows():
            index += 1 
            artist_name = row['artist_name']
            song_name = row['track_name']
            # print("\n")
            # print(f"Working on track {i}, {song_name} by {artist_name}.")
            artist = genius.search_artist(row['artist_name'], max_songs=0)
            try: 
                song = artist.song(row['track_name'])
            except AttributeError:
                # print(f'{artist_name} does not exist on genius')
                artists_not_found += 1
                continue
            try:
                lyrics = re.sub(r'[\(\[].*?[\)\]]', '', song.lyrics)
            except AttributeError:
                # print(f'{song_name} does not exist on genius')
                songs_not_found += 1
                continue
            #remove empty lines
            lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
            songs_found += 1 
#             response = GetLyrics.request_song_info(self, row['artist_name'], row['track_name'])
#             remote_song_info = GetLyrics.check_hits(self)
#             if remote_song_info == None:
#                 lyrics = None
#                 print(f"Track {i} is not in the Genius database.")
#             else:
#                 url = GetLyrics.get_url(self)
#                 lyrics = GetLyrics.scrape_lyrics(self)
#                 if lyrics == None:
#                     print(f"Track {i} is not in the Genius database.")
#                 else:
#                     print(f"Retrieved track {i} lyrics!")
            if index % 10 == 0:
                time_dif = time() - start_time
                songs_per_second = index / time_dif
                success_rate = songs_found / index
                print(f'{songs_found} songs found, {artists_not_found} artists not found, {songs_not_found} songs not found, {index} songs processed in {time_dif} seconds, rate: {songs_per_second}, success_rate: {success_rate}')
            # song_lyrics.append(lyrics)
        return song_lyrics

In [ ]:
x = GetLyrics(df1_sample).get_lyrics()

6 songs found, 2 artists not found, 2 songs not found, 10 songs processed in 8.729602575302124 seconds, rate: 1.145527521297716, success_rate: 0.6
11 songs found, 2 artists not found, 7 songs not found, 20 songs processed in 17.392635583877563 seconds, rate: 1.1499119787537768, success_rate: 0.55
18 songs found, 3 artists not found, 9 songs not found, 30 songs processed in 26.382872581481934 seconds, rate: 1.1371013488901478, success_rate: 0.6
22 songs found, 5 artists not found, 13 songs not found, 40 songs processed in 34.04745173454285 seconds, rate: 1.1748309480505994, success_rate: 0.55
29 songs found, 5 artists not found, 16 songs not found, 50 songs processed in 45.76179814338684 seconds, rate: 1.0926144082742, success_rate: 0.58
51 songs found, 9 artists not found, 20 songs not found, 80 songs processed in 77.91995787620544 seconds, rate: 1.0266946002088349, success_rate: 0.6375
58 songs found, 9 artists not found, 23 songs not found, 90 songs processed in 90.38530445098877 sec

In [ ]:
x